<a href="https://colab.research.google.com/github/LuthoYRN/2048-Game/blob/master/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 Urban Sound Classification using Deep Learning

**Project:** CSC4025Z Assignment 2 - Neural Network Application  
**Dataset:** Urban Sound 8K (8,732 audio samples, 10 classes)  
**Task:** Multi-class audio classification  

---

## Project Overview

This notebook implements and compares two approaches for urban sound classification:

1. **Baseline:** Naive Bayes Classifier
2. **Neural Network:** Artificial Neural Network (ANN)

---

## Dataset: Urban Sound 8K

**Classes (10):**
1. air_conditioner
2. car_horn
3. children_playing
4. dog_bark
5. drilling
6. engine_idling
7. gun_shot
8. jackhammer
9. siren
10. street_music

**Samples:** 8,732 audio files (4 seconds each)  

# Setting Up



In [1]:
import torch
import sys

print("="*70)
print("SYSTEM INFORMATION")
print("="*70)

# Check Python version
print(f"Python version: {sys.version}")

# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Check CUDA availability
if torch.cuda.is_available():
    print(f"GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    device = torch.device('cuda')
else:
    print("WARNING: GPU not available, using CPU")
    device = torch.device('cpu')

print(f"\nUsing device: {device}")
print("="*70)

SYSTEM INFORMATION
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch version: 2.8.0+cu126
GPU Available: Tesla T4
CUDA Version: 12.6
Number of GPUs: 1

Using device: cuda


In [2]:
print("="*70)
print("INSTALLING REQUIRED LIBRARIES")
print("="*70)

# Install audio processing libraries
!pip install -q librosa soundfile

# Verify installations
import librosa
import soundfile as sf

print(f"librosa version: {librosa.__version__}")
print("soundfile installed successfully")
print("="*70)
print("All libraries installed!")
print("="*70)

INSTALLING REQUIRED LIBRARIES
librosa version: 0.11.0
soundfile installed successfully
All libraries installed!


In [3]:
from google.colab import drive
import os

print("="*70)
print("MOUNTING GOOGLE DRIVE")
print("="*70)

# Mount Google Drive
drive.mount('/content/drive')

# Create project directory
PROJECT_DIR = '/content/drive/MyDrive/UrbanSound_Project'
os.makedirs(PROJECT_DIR, exist_ok=True)

# Create subdirectories
os.makedirs(f'{PROJECT_DIR}/data', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/models', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/results', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/figures', exist_ok=True)

print(f"\nGoogle Drive mounted successfully!")
print(f"Project directory: {PROJECT_DIR}")
print(f"\nCreated subdirectories:")
print(f"   - {PROJECT_DIR}/data")
print(f"   - {PROJECT_DIR}/models")
print(f"   - {PROJECT_DIR}/results")
print(f"   - {PROJECT_DIR}/figures")
print("="*70)

MOUNTING GOOGLE DRIVE
Mounted at /content/drive

Google Drive mounted successfully!
Project directory: /content/drive/MyDrive/UrbanSound_Project

Created subdirectories:
   - /content/drive/MyDrive/UrbanSound_Project/data
   - /content/drive/MyDrive/UrbanSound_Project/models
   - /content/drive/MyDrive/UrbanSound_Project/results
   - /content/drive/MyDrive/UrbanSound_Project/figures


In [4]:
print("="*70)
print("IMPORTING LIBRARIES")
print("="*70)

# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import json
import pickle
from pathlib import Path

# Audio processing
import librosa
import librosa.display
import soundfile as sf

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Seting random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("All libraries imported successfully!")
print(f"Random seed set to: {RANDOM_SEED}")
print(f"Device: {device}")
print("="*70)

IMPORTING LIBRARIES
All libraries imported successfully!
Random seed set to: 42
Device: cuda


In [5]:
print("="*70)
print("GLOBAL CONFIGURATION")
print("="*70)

# Configuration dictionary
CONFIG = {
    # Paths
    'project_dir': PROJECT_DIR,
    'data_dir': f'{PROJECT_DIR}/data',
    'models_dir': f'{PROJECT_DIR}/models',
    'results_dir': f'{PROJECT_DIR}/results',
    'figures_dir': f'{PROJECT_DIR}/figures',

    # Audio parameters
    'sample_rate': 22050,
    'duration': 4,
    'n_mfcc': 120,  # Based on Barua et al. (2023)

    # Dataset parameters
    'n_classes': 10,
    'class_names': [
        'air_conditioner',
        'car_horn',
        'children_playing',
        'dog_bark',
        'drilling',
        'engine_idling',
        'gun_shot',
        'jackhammer',
        'siren',
        'street_music'
    ],

    # Training parameters
    'test_size': 0.2,
    'batch_size': 400,  # Based on Barua et al. (2023)
    'learning_rate': 0.001, #Start value
    'num_epochs': 100,  # Start value
    'early_stopping_patience': 20,

    # Model architecture (Based on Barua et al. 2023)
    'hidden_layers': [1000, 750, 500, 250, 100],
    'dropout_rate': 0.5,

    # Random seed
    'random_seed': RANDOM_SEED,
    'device': str(device)
}

# Save configuration
with open(f"{CONFIG['results_dir']}/config.json", 'w') as f:
    json.dump(CONFIG, f, indent=4)

# Display configuration
print("\nConfiguration:")
for key, value in CONFIG.items():
    if key != 'class_names':  # Skip long list
        print(f"   {key}: {value}")
    else:
        print(f"   {key}: {len(value)} classes")

print(f"\nConfiguration saved to: {CONFIG['results_dir']}/config.json")
print("="*70)

GLOBAL CONFIGURATION

Configuration:
   project_dir: /content/drive/MyDrive/UrbanSound_Project
   data_dir: /content/drive/MyDrive/UrbanSound_Project/data
   models_dir: /content/drive/MyDrive/UrbanSound_Project/models
   results_dir: /content/drive/MyDrive/UrbanSound_Project/results
   figures_dir: /content/drive/MyDrive/UrbanSound_Project/figures
   sample_rate: 22050
   duration: 4
   n_mfcc: 120
   n_classes: 10
   class_names: 10 classes
   test_size: 0.2
   batch_size: 400
   learning_rate: 0.001
   num_epochs: 100
   early_stopping_patience: 20
   hidden_layers: [1000, 750, 500, 250, 100]
   dropout_rate: 0.5
   random_seed: 42
   device: cuda

Configuration saved to: /content/drive/MyDrive/UrbanSound_Project/results/config.json
